In [ ]:
import subprocess
import sys

# Install required libraries
!pip install datasets
!pip install accelerate

# Import necessary libraries
import pandas as pd
import torch
from nltk.tokenize import TweetTokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Verify versions
import transformers
import accelerate
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


2024-06-05 20:13:35.902577: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Transformers version: 4.39.2
Accelerate version: 0.28.0


In [ ]:
'''from google.colab import drive
import shutil
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/final_jawn.csv'
destination_path = '/content/sample_data/'
shutil.copy(file_path, destination_path)'''

df = pd.read_csv('/home/rfoley/private/JAWNDCI.csv')

# Assuming the DataFrame has columns 'text' and 'nominate_dim1' for the ideological scores
# Select only the required columns
sdf = df[['Subject', 'nominate_dim1']]
sdf.columns = ['text','nominate_dim1']
print(sdf.head())
bdf = df[['Body', 'nominate_dim1']]
bdf.columns = ['text', 'nominate_dim1']

                                                text  nominate_dim1
0  E-Alert: Rogers: House-Passed NDAA Supports ou...          0.377
1  eAlert: Rogers Praises Supreme Court's Decisio...          0.377
2  eAlert: Rogers Reintroduces Legislation to Pro...          0.377
3   We Must Secure our Border to Keep Americans Safe          0.377
4  eNewsletter: Biden's Socialist Policies are Hu...          0.377


In [ ]:
# Check for missing values
if sdf.isnull().values.any():
    print("Warning: Missing values detected in the dataset. Removing missing values...")
    sdf = sdf.dropna()
print(sdf.head())
if bdf.isnull().values.any():
    print("Warning: Missing values detected in the dataset. Removing missing values...")
    bdf = bdf.dropna()

                                                text  nominate_dim1
0  E-Alert: Rogers: House-Passed NDAA Supports ou...          0.377
1  eAlert: Rogers Praises Supreme Court's Decisio...          0.377
2  eAlert: Rogers Reintroduces Legislation to Pro...          0.377
3   We Must Secure our Border to Keep Americans Safe          0.377
4  eNewsletter: Biden's Socialist Policies are Hu...          0.377


In [ ]:
# Convert the DataFrame to a Hugging Face Dataset
sdataset = Dataset.from_pandas(sdf)
bdataset = Dataset.from_pandas(bdf)

# Split the dataset into train and validation sets
sdataset = sdataset.train_test_split(test_size=0.2)
s_train_dataset = sdataset['train']
s_eval_dataset = sdataset['test']

bdataset = bdataset.train_test_split(test_size=0.2)
b_train_dataset = bdataset['train']
b_eval_dataset = bdataset['test']

# Load Roberta tokenizer
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
def tokenize_function(examples):
    encoded = roberta_tokenizer(examples['text'], is_split_into_words=False, padding='max_length', truncation=True)
    return encoded

In [ ]:
print(s_train_dataset)

Dataset({
    features: ['text', 'nominate_dim1', '__index_level_0__'],
    num_rows: 99916
})


In [ ]:
# Apply tokenization to the datasets
s_train_dataset = s_train_dataset.map(tokenize_function, batched=True)
s_eval_dataset = s_eval_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
s_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'nominate_dim1'])
s_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'nominate_dim1'])

# Rename 'nominate_dim1' to 'labels' because Trainer expects labels column
s_train_dataset = s_train_dataset.rename_column("nominate_dim1", "labels")
s_eval_dataset = s_eval_dataset.rename_column("nominate_dim1", "labels")

# Ensure dataset columns are in the correct format for the model
s_train_dataset = s_train_dataset.map(lambda examples: {'labels': examples['labels'].float()}, batched=True)
s_eval_dataset = s_eval_dataset.map(lambda examples: {'labels': examples['labels'].float()}, batched=True)

Map:   0%|          | 0/99916 [00:00<?, ? examples/s]

Map:   0%|          | 0/24979 [00:00<?, ? examples/s]

Map:   0%|          | 0/99916 [00:00<?, ? examples/s]

Map:   0%|          | 0/24979 [00:00<?, ? examples/s]

In [ ]:
# Apply tokenization to the datasets
b_train_dataset = b_train_dataset.map(tokenize_function, batched=True)
b_eval_dataset = b_eval_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch tensors
b_train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'nominate_dim1'])
b_eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'nominate_dim1'])

# Rename 'nominate_dim1' to 'labels' because Trainer expects labels column
b_train_dataset = b_train_dataset.rename_column("nominate_dim1", "labels")
b_eval_dataset = b_eval_dataset.rename_column("nominate_dim1", "labels")

# Ensure dataset columns are in the correct format for the model
b_train_dataset = b_train_dataset.map(lambda examples: {'labels': examples['labels'].float()}, batched=True)
b_eval_dataset = b_eval_dataset.map(lambda examples: {'labels': examples['labels'].float()}, batched=True)

Map:   0%|          | 0/99916 [00:00<?, ? examples/s]

Map:   0%|          | 0/24979 [00:00<?, ? examples/s]

Map:   0%|          | 0/99916 [00:00<?, ? examples/s]

Map:   0%|          | 0/24979 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
#https://www.google.com/search?q=how+to+use+less+disk+space+when+training+a+pretrained+model&oq=how+to+use+less+disk+space+when+training+a+pretrained+model&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIHCAEQIRigATIHCAIQIRigATIHCAMQIRigATIHCAQQIRigATIHCAUQIRirAtIBCTIwOTE4ajBqNKgCALACAQ&sourceid=chrome&ie=UTF-8
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',  # Use 'evaluation_strategy' if 'eval_strategy' doesn't work
    learning_rate=2e-5, #look into this
    per_device_train_batch_size=4,  # Reduce batch size for faster computation
    per_device_eval_batch_size=4,
    save_steps=50000,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging_dir='./logs', #maybe get rid of logging
    #logging_steps=10,
    fp16=True,  # Enable mixed precision training
)

# Roberta model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=1)  # Regression output

# Define Trainers
s_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=s_train_dataset,
    eval_dataset=s_eval_dataset,
)

b_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=b_train_dataset,
    eval_dataset=b_eval_dataset,
)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to 

In [ ]:
#SUBJECT
# Train the model
s_trainer.train()

# Evaluate the model
s_eval_results = s_trainer.evaluate()
print(f"Subject line evaluation results: {eval_results}")

# Save the model
model.save_pretrained('./s_trained_model')
roberta_tokenizer.save_pretrained('./s_trained_model')

Epoch,Training Loss,Validation Loss


In [ ]:
# Get predictions
predictions = s_trainer.predict(eval_dataset)

# The predictions object contains several elements including predictions, label_ids, and metrics
preds = predictions.predictions
predicted_values = preds.squeeze()

In [ ]:
# Calculate regression metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_recall_fscore_support, roc_auc_score

mse = mean_squared_error(s_eval_dataset['labels'], predicted_values)
mae = mean_absolute_error(s_eval_dataset['labels'], predicted_values)
r2 = r2_score(s_eval_dataset['labels'], predicted_values)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

accuracy = accuracy_score(s_eval_dataset['labels'], predicted_values)
precision, recall, f1, _ = precision_recall_fscore_support(s_eval_dataset['labels'], predicted_values, average='binary')
roc_auc = roc_auc_score(s_eval_dataset['labels'], predicted_values)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"ROC-AUC: {roc_auc}")